In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
df = pd.read_csv('synthetic_gmv_data_1.2.csv')
df

,user_id,gmv,group_name
0,myo4ixol31,1428,test
1,myo4ixol31,1428,test
2,myo4ixol31,1071,test
3,myo4ixol31,1071,test
4,pkzf2889ww,351,test
...,...,...,...
799056,egoeez9fs9,712,control
799057,egoeez9fs9,641,control
799058,egoeez9fs9,855,control
799059,da9kujvymq,1110,control


In [3]:
data = df.groupby(['user_id', 'group_name'], as_index=False).agg({'gmv': 'sum', 'user_id':'count'})
data

,group_name,gmv,user_id
0,control,733,1
1,test,3187,3
2,control,2933,6
3,test,1695,5
4,control,1496,4
...,...,...,...
196386,test,1141,2
196387,control,2364,4
196388,test,1923,4
196389,control,1235,2


In [8]:
a = data.loc[data.group_name == 'control']
b = data.loc[data.group_name == 'test']

In [4]:
def safe_divide(x, y):
    try:
        return x / y
    except ZeroDivisionError:
        return np.nan

## Линеарализация

In [5]:
def linearization_type_2(x_num, x_denom, y_num, y_denom):
    n = len(x_num)
    m = len(y_num)
    x_num_bar = np.mean(x_num)
    y_num_bar = np.mean(y_num)
    x_denom_bar = np.mean(x_denom)
    y_denom_bar = np.mean(y_denom)
    x_estimator = safe_divide(x_num_bar, x_denom_bar)
    y_estimator = safe_divide(y_num_bar, y_denom_bar)
    delta_estimator = x_estimator - y_estimator
    x_linear = x_estimator  + safe_divide(1, x_denom_bar)*(np.array(x_num) - x_estimator*np.array(x_denom))
    y_linear = y_estimator  + safe_divide(1, y_denom_bar)*(np.array(y_num) - y_estimator*np.array(y_denom))
    t_stat, p_value = stats.ttest_ind(x_linear, y_linear, equal_var=False)
    return t_stat, p_value

In [9]:
t, pvalue = linearization_type_2(a.gmv, a.user_id, b.gmv, b.user_id)

In [10]:
round(t, 3), round(pvalue, 3)

(-2.344, 0.019)